### Tokenizer Test

In [1]:
# !pip install transformers
from transformers import AutoTokenizer

prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids
for t in input_ids:
    print(t,"\t:", tokenizer.decode(t))


2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


### AutoModelForCausalLM test
- 위 예시와 아래의 예시에서 AutoTokenizer 와 AutoModelForCausalLM 을 사용했다는 점에 주목합시다.
- transformers 라이브러리에서는 수백 개의 모델과 대응하는 토크나이저를 지원합니다. AutoTokenizer 와 AutoModelFor* 을 활용해 다양한 모델을 쉽게 로드해 사용할 수 있습니다.
- 단, 무슨 모델은 불러올지는 명시해야합니다. 아래처럼 Qwen2 0.5b 모델을 사용할때는 AutoModelForCausalLM, 분류 모델을 불러올때는 AutoModelForSequenceClassification, 객체 탐지에는 AutoModelForObjectDetection을 사용할 수 있습니다. 각 라이브러리 명을 보면 대략적으로 무슨 모델을 로드할때 사용할 지 판단할 수 있습니다.

In [2]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

# 초크나이저를 다시 호출하되, pytorch 텐서를 반환하도록 지정
# 모델이 정수 리스트 대신 파이토치 텐서를 입력으로 받음

input_ids = tokenizer(prompt, return_tensors='pt').input_ids
output = model(input_ids)
output.logits.shape


torch.Size([1, 7, 151936])

- 출력의 첫 번째 차원은 배치 크기를 나타내고 우리는 한 문장만 전달 했기에 그 크기는 1입니다.
- 두 번째 차원은 문장의 길이, 입력 문장의 토큰 수를 나타냅니다. 토크나이저마다 문장을 slice하는 기준이 다르기에 그 값은 같은 문장이라도 조금씩 다를 수 있습니다.
- 세 번째 차원은 어휘 사전 크기를 나타냅니다.
- 이러한 값들은 어휘 사전의 토큰에 대응하는 모델의 초기 출력 값인 logit으로 [0.1, -2.1, 1.2, 0.01 ...] 같은 숫자 리스트입니다. 이 logit을 사용해 다음 이어질 확률이 가장 높은 토큰을 선택할 수 있고 logit을 확률로 변환하는 방법도 있습니다.

In [3]:
final_logits = model(input_ids).logits[0,-1]
final_logits.argmax()

tensor(3729)

- 3729는 모델이 'It was a dark and stormy'라는 문장을 보고 다음에 올 가능성이 가장 높은 단어를 나타내는 일종의 ID값입니다. 이를 decode하면 ' night'라는 단어가 나오며 이를 통해 모델이 다음 흐름을 어떤 단어로 학습하고 산출했는지 알 수 있습니다.

In [4]:
len(final_logits)

151936

- 전체 151936개 중 가장 큰 값임을 알 수 있습니다.

In [5]:
tokenizer.decode(final_logits.argmax())

' night'

- 상위 10개의 확률 분포를 볼 수 있습니다. 전체 어휘 사전의 확률을 모두 더하면 100이 됩니다. 모든 어휘가 확률을 가지고 있으나 상위를 제외하고는 매우 낮습니다.

In [6]:
import torch
# top10_logits = torch.topk(final_logits,10)
# for i in top10_logits.indices:
#     print(tokenizer.decode(i))

top10 = torch.topk(final_logits.softmax(dim=0),10)
for v,i in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(i):<10} {v.item():.3%}")

 night     88.708%
 evening   4.300%
 day       2.192%
 morning   0.487%
 winter    0.449%
 afternoon 0.272%
 Saturday  0.252%
 Sunday    0.187%
 Friday    0.171%
 October   0.165%


### 단어 변경하기, 입력 문자열 변경하기, 문법 오류 실험해보기

In [7]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


def test_process(prompt:str):
    prompt = prompt
    # tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
    # input_ids = tokenizer(prompt).input_ids
    # input_ids
    # for t in input_ids:
    #     print(t,"\t:", tokenizer.decode(t))
        

    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
        
    final_logits = model(input_ids).logits[0,-1]
    final_logits.argmax()

    top10 = torch.topk(final_logits.softmax(dim=0),10)
    for v,i in zip(top10.values, top10.indices):
        print(f"{tokenizer.decode(i):<10} {v.item():.3%}")

case = ["It was a stormy and dark", "What's wrong with you? Why", "It were and and and"]

for c in case:
    print(c)
    test_process(c)
    print()

It was a stormy and dark
 night     67.609%
 day       6.383%
 evening   6.228%
 morning   2.221%
 summer    1.482%
 afternoon 1.437%
 winter    1.432%
 Saturday  0.931%
 weekend   0.827%
 Sunday    0.788%

What's wrong with you? Why
 are       27.436%
 do        15.871%
 don       12.520%
 can       6.287%
 did       5.347%
 didn      3.023%
 aren      2.746%
 you       2.705%
 not       2.338%
 have      1.965%

It were and and and
 and       27.702%
 I         4.080%
 the       3.245%

          2.408%
 a         1.815%
 ,         1.733%
 it        1.629%
,          1.543%
 .         1.424%
 that      1.393%

